In [35]:
pip install catboost

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [37]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import joblib


In [39]:
train_dir = r'C:\Users\IT SHOP\Desktop\Alzheimer_s Dataset\train'
test_dir = r'C:\Users\IT SHOP\Desktop\Alzheimer_s Dataset\test'

In [41]:
input_shape = (128, 128, 3)
batch_size = 32

In [43]:
data_generator = ImageDataGenerator(rescale=1./255)

In [15]:
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 5121 images belonging to 4 classes.


In [17]:
test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 1279 images belonging to 4 classes.


In [19]:
model = Sequential([
    Conv2D(32, (3, 3), input_shape=input_shape),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64),
    Activation('relu'),
    Dropout(0.5)
])

C:\Users\IT SHOP\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
train_features = model.predict(train_generator)
train_features = train_features.reshape(train_features.shape[0], -1)  # Flatten features for classifier
train_labels = train_generator.classes

  1/161 ━━━━━━━━━━━━━━━━━━━━ 20s 125ms/step

C:\Users\IT SHOP\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


161/161 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step


In [23]:
test_features = model.predict(test_generator)
test_features = test_features.reshape(test_features.shape[0], -1)
test_labels = test_generator.classes

40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step


In [25]:
cb_classifier = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6)
cb_classifier.fit(train_features, train_labels)

0:	learn: 1.3210257	total: 149ms	remaining: 2m 28s
1:	learn: 1.2682466	total: 164ms	remaining: 1m 21s
2:	learn: 1.2233193	total: 178ms	remaining: 59.1s
3:	learn: 1.1865898	total: 194ms	remaining: 48.3s
4:	learn: 1.1513211	total: 211ms	remaining: 42s
5:	learn: 1.1219765	total: 226ms	remaining: 37.5s
6:	learn: 1.0973009	total: 242ms	remaining: 34.3s
7:	learn: 1.0754344	total: 258ms	remaining: 32s
8:	learn: 1.0549466	total: 273ms	remaining: 30s
9:	learn: 1.0366285	total: 289ms	remaining: 28.6s
10:	learn: 1.0202427	total: 304ms	remaining: 27.4s
11:	learn: 1.0051788	total: 377ms	remaining: 31s
12:	learn: 0.9918206	total: 401ms	remaining: 30.4s
13:	learn: 0.9795095	total: 420ms	remaining: 29.6s
14:	learn: 0.9666338	total: 434ms	remaining: 28.5s
15:	learn: 0.9557416	total: 448ms	remaining: 27.6s
16:	learn: 0.9439456	total: 462ms	remaining: 26.7s
17:	learn: 0.9347204	total: 476ms	remaining: 26s
18:	learn: 0.9249140	total: 490ms	remaining: 25.3s
19:	learn: 0.9162148	total: 504ms	remaining: 24.7

In [45]:
test_predictions = cb_classifier.predict(test_features)

In [47]:
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Accuracy: 55.82%


In [49]:
model.save('cnn_feature_extractor6.h5')

In [33]:
joblib.dump(cb_classifier, 'catboost_classifier.pkl')

['catboost_classifier.pkl']

In [51]:
from keras.models import load_model
import joblib
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the saved models
cnn_model = load_model('cnn_feature_extractor9.h5')
xg_classifier = joblib.load('xgb_classifier.pkl')

def classify_image(img_path):
    # Load and preprocess the new image
    img = image.load_img(img_path, target_size=(128, 128))  # Resize to match the CNN input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to match the training preprocessing

    # Extract features using the CNN model
    features = cnn_model.predict(img_array)
    features = features.reshape(1, -1)  # Flatten to 1D for xgboost

    # Classify the features with the xgboost model
    prediction = xg_classifier.predict(features)
    
    # Interpret the prediction
    class_labels = {0: 'MildDemented', 1: 'ModerateDemented',2: 'NonDemented',3:'VeryMildDemented'}  # Adjust based on your dataset labels
    result = class_labels[prediction[0]]
    return result

# Test the function with a new image
img_path = r'C:\Users\IT SHOP\Desktop\Alzheimer_s Dataset\test\NonDemented\26 (86).jpg'
result = classify_image(img_path)
print(f"The image is classified as: {result}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
The image is classified as: NonDemented
